## Gathering the data -- using Google BigQuery

Process:
    
1. create a project and project dataset in BigQuery
2. Query big data (see three used in this project queries below)
3. save/copy the query results into a table
4. export the dataset table into a csv file

-- QUERY 1
-- What is the most common factor in a motor vehicle collision in Brooklyn? 
-- This query counts the number of collisions for each collision type, in descending order.

SELECT
  contributing_factor_vehicle_1 AS collision_factor,
  COUNT(*) num_collisions
FROM
  `bigquery-public-data.new_york.nypd_mv_collisions`
WHERE
  contributing_factor_vehicle_1 != "Unspecified"
  AND contributing_factor_vehicle_1 != ""
  AND borough = "BROOKLYN"
  AND cast(timestamp as date) between "2014-01-01" AND "2017-12-31"
GROUP BY
  1
ORDER BY
  num_collisions DESC

-- QUERY 2
-- What are the most dangerous streets for motor vehicle collisions in Brooklyn? 
-- This query counts the number of fatalities and injured by streets. 
#standardSQL
SELECT
  on_street_name,
  SUM(number_of_persons_killed) AS deaths,
  SUM(number_of_persons_injured) AS injuries
FROM
  `bigquery-public-data.new_york.nypd_mv_collisions`
WHERE
  on_street_name <> ''
  AND borough = "BROOKLYN"
  AND cast(timestamp as date) between "2014-01-01" AND "2017-12-31"
GROUP BY
  on_street_name
ORDER BY
  deaths DESC
LIMIT
  10


-- QUERY 3
-- What is the most common factor in a motor vehicle collision in NYC's Brooklyn? and what other details can we garner from Brooklyn incidents when looking at primary collision factors? 
-- This query counts the number of fatalities by streets. 


-- check for weird spelling of Brooklyn
#standardSQL
SELECT
  DISTINCT(borough)
FROM
  `bigquery-public-data.new_york.nypd_mv_collisions`

SELECT
  *
FROM
  `bigquery-public-data.new_york.nypd_mv_collisions`
WHERE
  borough = "BROOKLYN"
  AND cast(timestamp as date) between "2014-01-01" AND "2017-12-31"

In [1]:
import os
from google.cloud import bigquery

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'linkedininsightscase-9ef4c28fb738.json'

In [2]:
client = bigquery.Client()

In [17]:
sql = """
SELECT *
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`
WHERE borough = "BROOKLYN"
AND cast(timestamp as date) between "2014-01-01" AND "2017-12-31"
ORDER BY timestamp ASC
"""

#pct_overlap_terms_by_days_apart.head()

In [18]:
query_job = client.query(sql).to_dataframe()
query_job.to_csv('bigquery_nypd_collisions.csv')

### Loading

In [6]:

## Loading in packages
import csv
import pandas as pd
import numpy as np #import the stats library from numpy
from scipy import stats
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib notebook
#%matplotlib inline
import seaborn as sns

import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

In [4]:
## read the csv pulled from the database
brooklyn_data = pd.read_csv('bigquery_nypd_collisions.csv')

,latitude,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,unique_key,zip_code
count,181254.000000,181254.000000,190906.000000,190906.000000,190906.000000,190906.000000,190906.000000,190906.000000,190906.000000,190906.000000,1.909060e+05,190902.000000
mean,40.653171,-73.946144,0.029784,0.000120,0.187653,0.000293,0.063314,0.000665,0.279164,0.001079,2.866000e+06,11219.208583
std,0.359062,0.650963,0.173469,0.010976,0.627147,0.017726,0.260368,0.025986,0.672540,0.033307,1.181275e+06,11.778004
min,0.000000,-74.040793,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.159660e+05,11201.000000
25%,40.631301,-73.979443,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.146321e+06,11209.000000
50%,40.658893,-73.953064,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.350936e+06,11218.000000
75%,40.682280,-73.926064,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.575585e+06,11230.000000
max,41.126150,0.000000,4.000000,1.000000,24.000000,2.000000,7.000000,2.000000,24.000000,2.000000,3.833854e+06,11421.000000


### Exploratory


In [7]:
brooklyn_data.head()

,borough,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,timestamp,latitude,longitude,...,number_of_persons_killed,off_street_name,on_street_name,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code
0,BROOKLYN,Unspecified,Unspecified,Unspecified,NaN,NaN,NORTH 3 STREET,2014-06-12T16:00:00.000Z,40.716384,-73.961423,...,0,NaN,BERRY STREET,352535,MOTORCYCLE,MOTORCYCLE,UNKNOWN,NaN,NaN,11249.0
1,BROOKLYN,Unspecified,NaN,NaN,NaN,NaN,BERRIMAN STREET,2017-12-18T07:00:00.000Z,40.676765,-73.880516,...,0,NaN,LIBERTY AVENUE,3812120,SPORT UTILITY / STATION WAGON,trail,NaN,NaN,NaN,11208.0
2,BROOKLYN,Lost Consciousness,NaN,NaN,NaN,NaN,STUYVESANT AVENUE,2014-01-17T08:30:00.000Z,40.688232,-73.933101,...,0,NaN,GATES AVENUE,184352,PASSENGER VEHICLE,AMBULANCE,NaN,NaN,NaN,11221.0
3,BROOKLYN,NaN,NaN,NaN,NaN,NaN,AVENUE J,2016-04-08T16:45:00.000Z,40.625389,-73.959502,...,0,NaN,EAST 17 STREET,3417441,PASSENGER VEHICLE,MOTORCYCLE,NaN,NaN,NaN,11230.0
4,BROOKLYN,Unspecified,NaN,NaN,NaN,NaN,BUSHWICK AVENUE,2015-03-28T20:00:00.000Z,40.705256,-73.939432,...,0,NaN,MCKIBBIN STREET,3194093,VAN,AMBULANCE,NaN,NaN,NaN,11206.0


In [11]:
brooklyn_data['timestamp'] = pd.to_datetime(brooklyn_data['timestamp'])


0        2014-06-12 16:00:00+00:00
1        2017-12-18 07:00:00+00:00
2        2014-01-17 08:30:00+00:00
3        2016-04-08 16:45:00+00:00
4        2015-03-28 20:00:00+00:00
                    ...           
190901   2014-11-19 07:30:00+00:00
190902   2017-05-11 06:30:00+00:00
190903   2017-01-09 23:20:00+00:00
190904   2014-10-29 17:00:00+00:00
190905   2015-01-07 15:58:00+00:00
Name: timestamp, Length: 190906, dtype: datetime64[ns, UTC]

In [12]:
brooklyn_data['hour'] = brooklyn_data['timestamp'].dt.hour

# Count the frequency of accidents for each hour
hour_freq = brooklyn_data['hour'].value_counts().sort_index()

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(hour_freq.index, hour_freq.values, marker='o', linestyle='-', color='blue')

# Set the x-axis labels
plt.xticks(hour_freq.index)

# Set the y-axis label
plt.ylabel('Accidents')

# Set the title
plt.title('Accidents by Time of Day')

# Display the plot
plt.tight_layout()
plt.show()


<IPython.core.display.Javascript object>